Test code here  

In [2]:
import requests
import pandas as pd 

In [4]:
#1
url = 'http://dataservices.imf.org/REST/SDMX_JSON.svc/'
key = 'CompactData/IFS/M.GB.PMP_IX' # adjust codes here

# Navigate to series in API-returned JSON data
data = (requests.get(f'{url}{key}').json()
        ['CompactData']['DataSet']['Series'])

# Create pandas dataframe from the observations
data_list = [[obs.get('@TIME_PERIOD'), obs.get('@OBS_VALUE')]
             for obs in data['Obs']]

df = pd.DataFrame(data_list, columns=['date', 'value'])
df = df.set_index(pd.to_datetime(df['date']))['value'].astype('float')

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [5]:
key = 'DataStructure/IFS'  # Method / series
dimension_list = requests.get(f'{url}{key}').json()\
            ['Structure']['KeyFamilies']['KeyFamily']\
            ['Components']['Dimension']
for n, dimension in enumerate(dimension_list):
    print(f'Dimension {n+1}: {dimension['@codelist']}')

Dimension 1: CL_FREQ
Dimension 2: CL_AREA_IFS
Dimension 3: CL_INDICATOR_IFS


In [6]:
def get_imf_code_list(dimension_list:list, dimension_num:int):
    url = 'http://dataservices.imf.org/REST/SDMX_JSON.svc/'
    #if dimension_num == 0, give codes for all elements of the dimension_list
    if dimension_num == 0:
        code_dict = {}
        for i in range(len(dimension_list)):
            search_key = dimension_list[i]['@codelist']
            key = f"CodeList/{search_key}"
            if key == "CodeList/CL_FREQ": #CL_FREQがなぜかErrorで返ってくる
                continue
            code_list = requests.get(f'{url}{key}').json()\
            ['Structure']['CodeLists']['CodeList']['Code']
            code = [(c['Description']['#text'],c['@value'])for c in code_list]
            code_dict[search_key] = code
        return code_dict
    else:
        search_key = dimension_list[dimension_num-1]['@codelist']
        key = f"CodeList/{search_key}"
        if key == "CodeList/CL_FREQ":
            return f"Error: Codes of CL_FREQ(dimension_num={dimension_num}) are invalid"
        else:
            code_list = requests.get(f'{url}{key}').json()\
            ['Structure']['CodeLists']['CodeList']['Code']
            code = [(c['Description']['#text'],c['@value']) for c in code_list]
            print(f"Retrieved {search_key}'s codes")
            return code

In [7]:
codes = get_imf_code_list(dimension_list, 0)

In [8]:
def get_imf_code(codes:list, string:str) -> str:
    dic = dict(codes)
    matching_keys = {key: value for key, value in dic.items() if string in key}
    return matching_keys

In [9]:
ls = get_imf_code(codes['CL_INDICATOR_IFS'], 'Gross Domestic Product')
ls

{'Gross Domestic Product, Deflator, Index': 'NGDP_D_IX',
 'Gross Domestic Product, Deflator, Seasonally Adjusted, Index': 'NGDP_D_SA_IX',
 'Gross Domestic Product, Nominal, Domestic Currency': 'NGDP_XDC',
 'Gross Domestic Product, Nominal, Seasonally Adjusted, Domestic Currency': 'NGDP_SA_XDC',
 'Gross Domestic Product, Nominal, Seasonally Adjusted, in Purchasing-Power-Parity International Dollars': 'NGDP_SA_PU',
 'Gross Domestic Product, Nominal, Unadjusted, Domestic Currency': 'NGDP_NSA_XDC',
 'Gross Domestic Product, Real, Domestic Currency': 'NGDP_R_XDC',
 'Gross Domestic Product, Real, Seasonally Adjusted, Domestic Currency': 'NGDP_R_SA_XDC',
 'Gross Domestic Product, Real, Seasonally Adjusted, in Purchasing-Power-Parity International Dollars': 'NGDP_R_SA_PU',
 'Gross Domestic Product, Real, Unadjusted, Domestic Currency': 'NGDP_R_NSA_XDC'}

---

In [10]:

# key includes two partners, B0 and W00 for EU and 
url = 'http://dataservices.imf.org/REST/SDMX_JSON.svc/'
key = 'CompactData/DOT/M.GB.TMG_CIF_USD.B0+W00'

# Retrieve data from IMF API
data = requests.get(f'{url}{key}').json()

# Convert results to pandas dataframe
df = pd.DataFrame({s['@COUNTERPART_AREA'] : {pd.to_datetime(i['@TIME_PERIOD']) : 
     round(float(i['@OBS_VALUE']), 1) for i in s['Obs']} 
     for s in data['CompactData']['DataSet']['Series']})

In [11]:
url = 'http://dataservices.imf.org/REST/SDMX_JSON.svc/'
key = 'GenericMetadata/IFS/M.GB.PMP_IX'
metadata = requests.get(f'{url}{key}').json()
country = metadata['GenericMetadata']['MetadataSet']\
            ['AttributeValueSet'][1]['ReportedAttribute']\
            [1]['ReportedAttribute'][3]['Value']['#text']
indicator = metadata['GenericMetadata']['MetadataSet']\
            ['AttributeValueSet'][2]['ReportedAttribute']\
            [1]['ReportedAttribute'][4]['Value']['#text']
print(f'Country: {country}; Indicator: {indicator}')

Country: United Kingdom; Indicator: Index


---

In [12]:
def get_imf_data(database, frequency, indicator, start, end):
    host = 'http://dataservices.imf.org/REST/SDMX_JSON.svc/CompactData'

    
    ds = database #IFS, BOP, etc
    
    freq = frequency
    
    ind = indicator
    
    url = host + '/' + ds + '/' + freq + '.' + ind + '.' + '?startPeriod=' + start + '&endPeriod=' + end

In [13]:
url = 'http://dataservices.imf.org/REST/SDMX_JSON.svc/'
key = 'CompactData/IFS/Q.JP.NGDP_R_XDC/?startPeriod=1980&endPeriod=2024' # adjust codes here

# Navigate to series in API-returned JSON data
data = (requests.get(f'{url}{key}').json())

print(data)


{'CompactData': {'@xmlns:xsi': 'http://www.w3.org/2001/XMLSchema-instance', '@xmlns:xsd': 'http://www.w3.org/2001/XMLSchema', '@xsi:schemaLocation': 'http://www.SDMX.org/resources/SDMXML/schemas/v2_0/message https://registry.sdmx.org/schemas/v2_0/SDMXMessage.xsd http://dataservices.imf.org/compact/IFS http://dataservices.imf.org/compact/IFS.xsd', '@xmlns': 'http://www.SDMX.org/resources/SDMXML/schemas/v2_0/message', 'Header': {'ID': '253bc872-1a92-4402-9af7-ce33c556d87c', 'Test': 'false', 'Prepared': '2025-04-01T18:37:25', 'Sender': {'@id': '1C0', 'Name': {'@xml:lang': 'en', '#text': 'IMF'}, 'Contact': {'URI': 'http://www.imf.org', 'Telephone': '+ 1 (202) 623-6220'}}, 'Receiver': {'@id': 'ZZZ'}, 'DataSetID': 'IFS'}, 'DataSet': {'@xmlns': 'http://dataservices.imf.org/compact/IFS'}}}


In [14]:
url = 'http://dataservices.imf.org/REST/SDMX_JSON.svc/'
key = 'CompactData/IFS/Q.US.NGDP_R_SA_PU/?startPeriod=1980&endPeriod=2024'
data = (requests.get(f'{url}{key}').json()['CompactData']['DataSet']['Series'])

# Create pandas dataframe from the observations
data_list = [[obs.get('@TIME_PERIOD'), obs.get('@OBS_VALUE')]
             for obs in data['Obs']]

df = pd.DataFrame(data_list, columns=['date', 'value'])
df = df.set_index(pd.to_datetime(df['date']))['value'].astype('float')
df

KeyError: 'Series'